In [1]:
import requests
import pandas as pd
from sqlalchemy import create_engine

In [2]:
url = "https://api.sampleapis.com/wines/reds"

data = requests.get(url).json()

# Raw data from wines API
data[:5]

[{'winery': 'Maselva',
  'wine': 'Emporda 2012',
  'rating': {'average': '4.9', 'reviews': '88 ratings'},
  'location': 'Spain\n·\nEmpordà',
  'image': 'https://images.vivino.com/thumbs/ApnIiXjcT5Kc33OHgNb9dA_375x500.jpg',
  'id': 1},
 {'winery': 'Ernesto Ruffo',
  'wine': 'Amarone della Valpolicella Riserva N.V.',
  'rating': {'average': '4.9', 'reviews': '75 ratings'},
  'location': 'Italy\n·\nAmarone della Valpolicella',
  'image': 'https://images.vivino.com/thumbs/nC9V6L2mQQSq0s-wZLcaxw_pb_x300.png',
  'id': 2},
 {'winery': 'Cartuxa',
  'wine': 'Pêra-Manca Tinto 1990',
  'rating': {'average': '4.9', 'reviews': '72 ratings'},
  'location': 'Portugal\n·\nAlentejo',
  'image': 'https://images.vivino.com/thumbs/L33jsYUuTMWTMy3KoqQyXg_pb_x300.png',
  'id': 3},
 {'winery': 'Schrader',
  'wine': 'Cabernet Sauvignon RBS Beckstoffer To Kalon Vineyard 2015',
  'rating': {'average': '4.9', 'reviews': '72 ratings'},
  'location': 'United States\n·\nOakville',
  'image': 'https://images.vivino.

In [3]:
df = pd.DataFrame(data)

# Dataframe before transformation
df

,winery,wine,rating,location,image,id
0,Maselva,Emporda 2012,"{'average': '4.9', 'reviews': '88 ratings'}",Spain\n·\nEmpordà,https://images.vivino.com/thumbs/ApnIiXjcT5Kc3...,1
1,Ernesto Ruffo,Amarone della Valpolicella Riserva N.V.,"{'average': '4.9', 'reviews': '75 ratings'}",Italy\n·\nAmarone della Valpolicella,https://images.vivino.com/thumbs/nC9V6L2mQQSq0...,2
2,Cartuxa,Pêra-Manca Tinto 1990,"{'average': '4.9', 'reviews': '72 ratings'}",Portugal\n·\nAlentejo,https://images.vivino.com/thumbs/L33jsYUuTMWTM...,3
3,Schrader,Cabernet Sauvignon RBS Beckstoffer To Kalon Vi...,"{'average': '4.9', 'reviews': '72 ratings'}",United States\n·\nOakville,https://images.vivino.com/thumbs/GpcSXs2ERS6ni...,4
4,Hundred Acre,Wraith Cabernet Sauvignon 2013,"{'average': '4.9', 'reviews': '68 ratings'}",United States\n·\nNapa Valley,https://images.vivino.com/thumbs/PBhGMcRNQ7aVn...,5
...,...,...,...,...,...,...
713,Dominus,Dominus (Christian Moueix) 2013,"{'average': '4.7', 'reviews': '548 ratings'}",United States\n·\nNapa Valley,https://images.vivino.com/thumbs/FvLdztxrRXOZx...,714
714,Château Cheval Blanc,Saint-Émilion Grand Cru (Premier Grand Cru Cla...,"{'average': '4.7', 'reviews': '546 ratings'}",France\n·\nSaint-Émilion Grand Cru,https://images.vivino.com/thumbs/AIWJqsLoSIyz0...,715
715,Masseto,Toscana 2011,"{'average': '4.7', 'reviews': '546 ratings'}",Italy\n·\nToscana,https://images.vivino.com/thumbs/GPvEr_IUSKS1C...,716
716,Soldera,Brunello di Montalcino Riserva 2005,"{'average': '4.7', 'reviews': '546 ratings'}",Italy\n·\nBrunello di Montalcino,https://images.vivino.com/thumbs/l_eXmV1KTdGzz...,717


In [4]:
# Seperating country and city from location column
df['country'] = df['location'].apply(lambda x: x.split("\n·\n")[0].strip())
df['city'] = df['location'].apply(lambda x: x.split("\n·\n")[1].strip() if "\n·\n" in x else None)

# seperating avg_rating and reviews from rating cloumn
ratings =[]
reviews =[]
for rating in data:
    ratings.append(float(rating["rating"]["average"]))
    reviews.append(int(rating["rating"]["reviews"].split()[0]))

df["avg_rating"] = ratings
df["total_reviews"] = reviews

# Deleting unnecessary columns
df.drop(columns =["location","rating"], inplace=True)

In [5]:
# Dataframe after transformation
df


,winery,wine,image,id,country,city,avg_rating,total_reviews
0,Maselva,Emporda 2012,https://images.vivino.com/thumbs/ApnIiXjcT5Kc3...,1,Spain,Empordà,4.9,88
1,Ernesto Ruffo,Amarone della Valpolicella Riserva N.V.,https://images.vivino.com/thumbs/nC9V6L2mQQSq0...,2,Italy,Amarone della Valpolicella,4.9,75
2,Cartuxa,Pêra-Manca Tinto 1990,https://images.vivino.com/thumbs/L33jsYUuTMWTM...,3,Portugal,Alentejo,4.9,72
3,Schrader,Cabernet Sauvignon RBS Beckstoffer To Kalon Vi...,https://images.vivino.com/thumbs/GpcSXs2ERS6ni...,4,United States,Oakville,4.9,72
4,Hundred Acre,Wraith Cabernet Sauvignon 2013,https://images.vivino.com/thumbs/PBhGMcRNQ7aVn...,5,United States,Napa Valley,4.9,68
...,...,...,...,...,...,...,...,...
713,Dominus,Dominus (Christian Moueix) 2013,https://images.vivino.com/thumbs/FvLdztxrRXOZx...,714,United States,Napa Valley,4.7,548
714,Château Cheval Blanc,Saint-Émilion Grand Cru (Premier Grand Cru Cla...,https://images.vivino.com/thumbs/AIWJqsLoSIyz0...,715,France,Saint-Émilion Grand Cru,4.7,546
715,Masseto,Toscana 2011,https://images.vivino.com/thumbs/GPvEr_IUSKS1C...,716,Italy,Toscana,4.7,546
716,Soldera,Brunello di Montalcino Riserva 2005,https://images.vivino.com/thumbs/l_eXmV1KTdGzz...,717,Italy,Brunello di Montalcino,4.7,546


In [6]:
# store Dataframe into a SQLlite Database with table name as 'wine_table'
disk_engine = create_engine("sqlite:///my_pipeline_database.db")
df.to_sql('wine_table',disk_engine, if_exists='replace')

718